Vérification des données
Import des librairies pandas, scikit-learn (module DecisionTreeRegressor)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly_express as px
import numpy as nu
from sklearn.tree import DecisionTreeRegressor

train = pd.read_csv('data/train.csv')

train.describe()
train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Visualisation de données pertinentes pour l'analyse :
- Répartition des âges
- Répartition des sexes
- Répartition des classes de passager
- Répartition des prix des billets

In [2]:
!pip install -U nbformat>=4.2.0

In [12]:
import nbformat

age = train['Age'].value_counts().reset_index()
age.columns = ['Age', 'Count']

fig = px.bar(age, x='Age', y='Count', title="Répartition des âges")

fig.show()

Sélection d'une donnée à modéliser
=> Colonne "Survived"

In [4]:
# Affichage des colonnes
print(train.columns)
# Nettoyage des données manquantes
# train = train.dropna(axis=0)
# Création d'une variable qui cible les données à analyser ('Survived')
y = train.Survived


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


Création d'un DataFrame (X) des données qui serviront à la prédiction
=> Pour ce premier test, je sélectionne les colonnes (features) 'Pclass', 'Sex', 'Age' et les stocke dans une variable 'train_features'
=> Création de la variable X contenant les features du DataFrame 'train'

Conversion des données str en données numérique (int)
=> L'arbre de décision traite uniquement des données numériques
=> Utilisation d'un module sklearn (LabelEncoder) pour convertir les données de
la colonne 'Sex'
Après conversion, les valeur 'Male' = 0 et 'Female' = 1


In [5]:
# Import du module LabelEncoder pour convertir la colonne 'Sex' en valeur numérique

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Sex'] = le.fit_transform(train['Sex'])

# Sélection des features
train_features = ['Pclass', 'Sex', 'Age']
#  Stockage dans la variable X
X = train[train_features]
# Verification des données

X.head()

,Pclass,Sex,Age
0,3,1,22.0
1,1,0,38.0
2,3,0,26.0
3,1,0,35.0
4,3,1,35.0


Utilisation d'un arbre de décision (DecisionTreeRegressor)
Stockage dans une variable (train_model)

In [6]:
# Définition du modèle 
train_model = DecisionTreeRegressor(random_state=1)
# Adaptation du modèle
train_model.fit(X, y)
# Affichage du score
train_model.score(X, y)







0.6509502882199665

Validation du modèle
=> Calcul de la moyenne absolue des erreurs (Mean Absolute Error)

In [7]:
# Appel du module mean_absolute_error 
from sklearn.metrics import mean_absolute_error
# Calcul de la moyenne des écarts entre les valeurs prédites et les valeurs des données réelles
prediction_train_model = train_model.predict(X)
mean_absolute_error(y, prediction_train_model)

0.16510503660975764

Utilisation de la librairie sklearn "train_test_split" pour vérifier les données
X = Données d'entrée (features ou les colonnes que l'on a choisi pour la prédiction)
y = Cible à prédire (target)
X_train = Données d'entrainement (features)
X_test = Données de test (features)
y_train = Cibles pour l'entrainement
y_test = Cibles pour le test
Le modèle est entraîné sur X_train et y_train puis testé sur X_test et y_test

In [8]:
# Import du module train_test_split
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=0)

# # Définition du modèle
prediction_train_model = DecisionTreeRegressor()
# # Adaptation du modèle
prediction_train_model.fit(train_X, train_y)
# # Obtention d'une valeur prédictive sur les données validées
val_predictions = prediction_train_model.predict(val_X)

print(mean_absolute_error(val_y, val_predictions))

0.2338968294275557


<span>Ce que je crois avoir compris :</span>
- Une fois qu'on a "entrainé" une première fois un modèle de données (train_model), on vérifie l'écart de prédiction entre les données fournies et ce que notre modèle a prédit: cet écart moyen est obtenu avec le Mean Absolute Error (MAE). 
- La phase suivante consiste à séparer les données pour les entrainer et les évaluer avec train_test_split, cela permet de vérifier si notre modèle fonctionne sur d'autres données

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error

def score_model(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    prediction = model.predict(val_X)
    return mean_absolute_error(val_y, prediction)

for max_leaf_nodes in [5, 25, 30, 40, 45, 50, 100, 250, 500]:
    mae = score_model(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print(f'Feuilles : {max_leaf_nodes} \t MAE : {mae}')

Feuilles : 5 	 MAE : 0.21787709497206703
Feuilles : 25 	 MAE : 0.2122905027932961
Feuilles : 30 	 MAE : 0.2011173184357542
Feuilles : 40 	 MAE : 0.2011173184357542
Feuilles : 45 	 MAE : 0.2011173184357542
Feuilles : 50 	 MAE : 0.18435754189944134
Feuilles : 100 	 MAE : 0.18435754189944134
Feuilles : 250 	 MAE : 0.18435754189944134
Feuilles : 500 	 MAE : 0.18435754189944134
